In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json, os

In [ ]:
models = [
    ("<path>/Experiments/prostatefAbl_none_10_1.0_16_OctreeNCASegmentation/", "OctreeNCA"),
    ("<path>/Experiments/prostate_m3d_M3dSegmentation/", "M3D-NCA"),
    ("<path>/Experiments/prostate_unet_UNetSegmentation/", "UNet"),
    ("<path>/Experiments/prostate_munet3d_UNet_resnet18_0_minUNet3DSegmentation/", "Resnet18"),
    ("<path>/Experiments/prostate_munet3d_UNet_densenet121_0_minUNet3DSegmentation/", "Densenet"),
]

In [3]:
annnotation_list = []
annotation_dict = {}
all_files = []
for i, (model, model_name) in enumerate(models):
    score = pd.read_csv(f"{model}/eval/standard.csv", sep='\t').loc[:, "DiceScore/0"] * 100

    score_std = score.std()
    score = score.mean()

    eval_file = {}
    mem_allocation = None
    if os.path.exists(f"{model}/mem_allocation.json"):
        mem_allocation = round(json.load(open(f"{model}/mem_allocation.json"))["byte"] / 1024**3, 2)
    num_params = json.load(open(f"{model}/num_params.json"))["num_params"]

    eval_file["model"] = model_name
    eval_file["mem_allocation"] = mem_allocation
    eval_file["num_params"] = num_params
    eval_file["score"] = round(score, 2)
    #eval_file["score_std"] = score_std
    df = pd.DataFrame(index=[i], data=eval_file)
    all_files.append(df)
    annnotation_list.append((num_params, score, model_name))
    annotation_dict[model_name] = (num_params, score)

all_files = pd.concat(all_files)

In [ ]:
def get_nnUNet_results(path: str, trainer: str) -> dict:
    FOLD = 0
    TRAIN_TASK = "Task506_Prostate_MEDSeg_split"
    EVAL_TASK = "Task508_Prostate_MEDSeg_split" 
    #both are the same dataset. However 506 is not properly preprocessed, so it cannot be used for evaluation
    # Later I preprocessed 508 correctly, which can also be used for evaluation. Future work should use 508 for training and evaluation

    eval_path = os.path.join(path,"evaluation", "nnUNet_ext", "3d_fullres",
                             TRAIN_TASK, TRAIN_TASK, f"{trainer}__nnUNetPlansv2.1", 
                             f"Generic_UNet/SEQ/head_None/fold_{FOLD}",EVAL_TASK,
                             "val_metrics_all.csv")
    results_path = os.path.join(path, "results", "nnUNet_ext", "3d_fullres",
                             TRAIN_TASK, TRAIN_TASK, f"{trainer}__nnUNetPlansv2.1", 
                             f"Generic_UNet/SEQ/fold_{FOLD}")
    results = pd.read_csv(eval_path, sep='\t')
    results = results[results["split"] == "test"]
    results = results[results["seg_mask"] == "mask_1"]
    results = results[results["metric"] == "Dice"]

    model_name = {
        "nnUNetTrainerNCA": "nnNCA",
        "nnUNetTrainerSequential": "nnUNet",
    }
    ret = {
        "model": model_name[trainer],
        "mem_allocation": round(json.load(open(os.path.join(results_path, "mem_allocation.json")))["byte"] / 1024**3, 2),
        "num_params": json.load(open(os.path.join(results_path, "num_params.json")))["num_params"],
        "score": results["value"].mean() * 100,
    }
    return ret

nnOctreeNCA = get_nnUNet_results("<path>/nnunet_nca", "nnUNetTrainerNCA")
nnUNet = get_nnUNet_results("<path>/nnunet_nca", "nnUNetTrainerSequential")
data = [nnUNet]
df = pd.DataFrame(data=data)
all_files = pd.concat([all_files, df])

In [5]:
from IPython.display import display, HTML
display(HTML(all_files.to_html()))

,model,mem_allocation,num_params,score
0,OctreeNCA,2.18,17600,82.510000
1,M3D-NCA,14.34,12480,80.520000
2,UNet,7.79,19071297,84.060000
3,Resnet18,4.30,42611121,85.620000
4,Densenet,6.20,31199985,84.760000
0,nnUNet,7.35,44797408,88.180718


In [7]:
all_files_styled = all_files.style.format(thousands=',', na_rep="-", precision=2)
all_files_styled.hide(axis="index")
all_files_styled.to_latex("tables/prostate.tex")